In [73]:
import time
from tinkerforge.ip_connection import IPConnection, Error
from tinkerforge.bricklet_performance_dc import BrickletPerformanceDC
from tinkerforge.bricklet_silent_stepper_v2 import BrickletSilentStepperV2

%matplotlib inline

HOST = "localhost"
PORT = 4223
PDC_UID = "SSV" 
SS_UID = "2akW"

# Create IP connection
ipcon = IPConnection()

# Create device objects
ss1 = BrickletSilentStepperV2(SS_UID, ipcon)
pdc = BrickletPerformanceDC(PDC_UID, ipcon)
SS_SPEED = 500

try:
    ipcon.connect(HOST, PORT)
    print("Connected to brickd")
except Error as e:
    print(f"Error connecting to brickd: {e}")


Connected to brickd


In [74]:
def initialize_motor(ss):
    # 416 mA, for 10W 24V system
    ss.set_motor_current(416) 
    # 1/8 steps (interpolated)
    ss.set_step_configuration(ss1.STEP_RESOLUTION_8, True) 
    ss.set_max_velocity(SS_SPEED) 
    ss.set_speed_ramping(SS_SPEED * 100, SS_SPEED * 100)
    ss.set_enabled(True) 

def turn_on_pump():
    pdc.set_drive_mode(pdc.DRIVE_MODE_DRIVE_COAST)  
    pdc.set_motion(32767, 32767)  # Optional: set max accel/decel just in case
    pdc.set_velocity(12000)       # 7K gives a nice gentle spray
    pdc.set_enabled(True)         # Enable motor

def turn_off_pump():
    pdc.set_velocity(0)           # Stop motor
    pdc.set_enabled(False)        # Cut power — acts like a relay off



In [ ]:
from datetime import datetime

# Every 10 minutes, mist for 5 seconds...
def start_experiment(
    duration=14 * 24 * 60 * 60, interval=15 * 60, misting_time=4
):
    start_time = time.time()

    initialize_motor(ss1)

    ss1.set_steps(100000)  # Drive 100K steps forward
    misting_iteration = 0

    while time.time() - start_time < duration:
        print('Misting iteration:', misting_iteration , )
        current_time = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
        print(f'Misting iteration: {misting_iteration} at {current_time}')
        print('starting misting!')
        turn_on_pump()
        time.sleep(misting_time)
        turn_off_pump()
        print('stopping misting!')
        # ss1.drive_forward()
        time.sleep(interval)


In [76]:
ML_PER_TWO_SEC_SPRAY = 5 # mL
interval = 15 * 60  # 10 minutes
total_mL_per_day = (24 * 60 * 60 / interval ) * ML_PER_TWO_SEC_SPRAY
print(f"Total mL per day: {total_mL_per_day} mL")

Total mL per day: 480.0 mL


In [77]:
# Every 6 spurts (for 2 seconds each) is 30 mL
# so each spurt is 5mL
start_experiment()

Misting iteration: 0
Misting iteration: 0 at 2025-05-08 19:57:49
starting misting!
stopping misting!
Misting iteration: 0
Misting iteration: 0 at 2025-05-08 19:57:58
starting misting!
stopping misting!
Misting iteration: 0
Misting iteration: 0 at 2025-05-08 19:58:07
starting misting!
stopping misting!


KeyboardInterrupt: 

In [ ]:
# Disconnect from brickd
ipcon.disconnect()
